In [2]:
import os
import openai
import tarfile
import replicate

In [3]:
def get_openai_key():
    # Check if the file exists
    if not os.path.isfile('../openai_key.txt'):
        # Create the file if it does not exist and write a default value or leave it blank
        with open('../openai_key.txt', 'w') as file:
            file.write('')  # You could prompt the user for a key or leave it blank
    # Read the key from the file
    with open('../openai_key.txt', 'r') as file:
        return file.read().strip()


# Usage
openai.api_key = get_openai_key()

In [4]:
def get_replicate_token():
    # Check if the file exists
    if not os.path.isfile('../replicate_token.txt'):
        # Create the file if it does not exist and write a default value or leave it blank
        with open('../replicate_token.txt', 'w') as file:
            file.write('')  # You could prompt the user for a key or leave it blank
    # Read the key from the file
    with open('../replicate_token.txt', 'r') as file:
        return file.read().strip()

os.environ["REPLICATE_API_TOKEN"] = get_replicate_token()

In [10]:

def extract_tar_files(tar_dir, output_dir):
    """
    Extracts all .tar files found in tar_dir to output_dir, skipping extraction if
    the output directory for the .tar file already exists.

    Parameters:
    - tar_dir: Directory containing .tar files.
    - output_dir: Directory where extracted files will be saved.
    """
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over each file in the tar_dir
    for file in os.listdir(tar_dir):
        if file.endswith(".tar"):
            # Construct full file path
            tar_path = os.path.join(tar_dir, file)
            # Construct the specific output path for this tar file
            specific_output_dir = os.path.join(output_dir, os.path.splitext(file)[0])
            
            # Check if the specific output directory already exists
            if not os.path.exists(specific_output_dir):
                # If not, create the directory and proceed with extraction
                os.makedirs(specific_output_dir)
                
                # Open the .tar file
                with tarfile.open(tar_path, "r:") as tar:
                    # Extract all contents to the specific output directory
                    tar.extractall(path=specific_output_dir)
                    print(f"Extracted {file} to {specific_output_dir}")
            else:
                # If directory exists, skip the extraction
                print(f"Skipping {file}, directory {specific_output_dir} already exists.")


extract_tar_files("../data/raw", "../data/raw")


Skipping juan-saenz.tar, directory ../data/raw/juan-saenz already exists.


In [6]:
import os
import ffmpeg
import os

def convert_mp4_to_mp3(directory):
    """
    Converts all MP4 files found in the given directory to MP3.

    Parameters:
    - directory: The directory to scan for MP4 files.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            # Construct the full path to the MP4 file
            full_path_to_mp4 = os.path.join(directory, filename)
            # Define the output path for the MP3 (same name, different extension)
            mp3_filename = filename[:-4] + '.mp3'
            full_path_to_mp3 = os.path.join(directory, mp3_filename)

            # Use ffmpeg to convert the MP4 to MP3
            try:
                ffmpeg.input(full_path_to_mp4).output(full_path_to_mp3).run(overwrite_output=True)
                print(f"Converted {filename} to {mp3_filename}")
            except ffmpeg.Error as e:
                print(f"Error converting file {filename}: {e}")

# Example usage: replace 'path_to_directory' with the path to your directory
convert_mp4_to_mp3('../data/raw/juan-saenz')




Converted writing-session-saenz.mp4 to writing-session-saenz.mp3


In [38]:
input = {
    "file": "/Users/tommasocalo/Documents/AutoInterviews/ai-interviews-processor/data/raw/juan-saenz/writing-session-saenz.mp3",
    "prompt": "Alberto Interviewin Juan",
    "file_url": "",
    "num_speakers": 2
    "language":"it"
}

output = replicate.run(
    "thomasmol/whisper-diarization:b9fd8313c0d492bf1ce501b3d188f945389327730773ec1deb6ef233df6ea119",
    input=input
)
print(output)